In [2]:
import joblib
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc, \
  ConfusionMatrixDisplay, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score, matthews_corrcoef
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv(r'..\..\10_fold_cross_validation\train_10folds.csv')
data

,MaxPartialCharge,FpDensityMorgan2,BCUT2D_CHGLO,BCUT2D_MRHI,PEOE_VSA12,PEOE_VSA6,SMR_VSA3,SlogP_VSA3,SlogP_VSA8,EState_VSA6,NumHAcceptors,NumSaturatedCarbocycles,fr_bicyclic,TARGET,Kfold
0,0.335201,1.714286,-2.072068,5.975550,0.000000,12.132734,9.551078,0.000000,38.974594,43.638476,6,0,1,1.0,0
1,0.226791,2.000000,-2.362642,7.150190,5.948339,0.000000,9.967957,0.000000,10.440599,11.336786,6,0,0,1.0,1
2,0.211302,1.772727,-2.089721,7.912349,9.837253,29.297126,4.983979,9.837253,10.902925,24.265468,4,0,1,0.0,1
3,0.226898,1.869565,-2.357502,6.433493,5.948339,35.334614,15.284746,0.000000,11.126903,12.263211,5,0,0,0.0,0
4,0.158370,1.870968,-2.421374,7.991366,0.000000,23.362825,19.935914,6.420822,0.000000,18.460054,9,0,1,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,0.152613,2.000000,-2.415835,6.432407,0.000000,35.334614,9.967957,11.343745,11.257379,6.066367,6,0,0,1.0,3
2735,0.335203,1.700000,-2.001320,6.164147,0.000000,30.331835,4.983979,0.000000,10.902925,36.528679,2,0,1,0.0,6
2736,0.264178,1.833333,-2.414068,5.737009,5.948339,38.112943,9.551078,11.784535,11.126903,11.614772,7,0,0,1.0,8
2737,0.338995,1.815789,-2.065301,7.218862,5.907180,59.014740,24.544948,18.386966,43.634305,6.196844,9,0,2,0.0,5


In [4]:
data.columns

Index(['MaxPartialCharge', 'FpDensityMorgan2', 'BCUT2D_CHGLO', 'BCUT2D_MRHI',
       'PEOE_VSA12', 'PEOE_VSA6', 'SMR_VSA3', 'SlogP_VSA3', 'SlogP_VSA8',
       'EState_VSA6', 'NumHAcceptors', 'NumSaturatedCarbocycles',
       'fr_bicyclic', 'TARGET', 'Kfold'],
      dtype='object')

In [5]:
def run(fold, data):
    # load the full training data with folds
    df = data
    # all columns are features except target and kfold columns
    features = [
        f for f in df.columns if f not in ("TARGET", "Kfold")
    ]
    # get training data using folds
    df_train = df[df.Kfold != fold].reset_index(drop=True)
    # get validation data using folds
    df_valid = df[df.Kfold == fold].reset_index(drop=True)
    # get training data
    X_train = df_train[features].values
    # get validation data
    X_valid = df_valid[features].values
    # initialize Logistic Regression model
    model = XGBClassifier(n_jobs = -1)
    model.fit(X_train, df_train.TARGET.values)
    valid_preds = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(df_valid.TARGET.values, valid_preds)
    y_pred = model.predict(X_valid)
    y_true = df_valid.TARGET.values
    accuracy = accuracy_score(y_true,y_pred)
    precision_1 = precision_score(y_true,y_pred,pos_label=1)
    precision_0 = precision_score(y_true,y_pred,pos_label=0)
    recall_1 = recall_score(y_true,y_pred,pos_label=1)
    recall_0 = recall_score(y_true,y_pred,pos_label=0)
    f1score = f1_score(y_true,y_pred)
    kappa = cohen_kappa_score(y_true,y_pred)
    MCC = matthews_corrcoef(y_true,y_pred)
    print(f"Fold = {fold}, AUC = {auc}, Accuracy = {accuracy}, \
          Precision_1 = {precision_1}, Precision_0 = {precision_0}\
          Recall_1 = {recall_1}, Recall_0 = {recall_0}, F1Score = {f1score}, kappa = {kappa}, MCC = {MCC}")
    
    return auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model

In [6]:
aucs, accuracies, precisions_1, precisions_0, recalls_1, recalls_0, f1scores, kappas, MCCs = [], [], [], [], [], [], [], [], []

for fold_ in range(10):
    auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model = run(fold_, data)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions_1.append(precision_1)
    precisions_0.append(precision_0)
    recalls_1.append(recall_1)
    recalls_0.append(recall_0)
    f1scores.append(f1score)
    kappas.append(kappa)
    MCCs.append(MCC)
    filename = 'XGBoost_' + str(fold_) + '.pkl'
    joblib.dump(model, filename)
print("\n")
print(f"Mean Scores: AUC = {np.mean(np.array(aucs))}, \
      Accuracy = {np.mean(np.array(accuracies))}, \
      Precision_1 = {np.mean(np.array(precisions_1))}, Precision_0 = {np.mean(np.array(precisions_0))}\
      Recall_1 = {np.mean(np.array(recalls_1))}, Recall_0 = {np.mean(np.array(recalls_0))}\
      F1Score = {np.mean(np.array(f1scores))} \
      Kappa = {np.mean(np.array(kappas))} \
      MCC = {np.mean(np.array(MCCs))}")

Fold = 0, AUC = 0.9653445162793407, Accuracy = 0.948905109489051,           Precision_1 = 0.9368421052631579, Precision_0 = 0.9553072625698324          Recall_1 = 0.9175257731958762, Recall_0 = 0.9661016949152542, F1Score = 0.9270833333333333, kappa = 0.8877640587512435, MCC = 0.8878781938279658
Fold = 1, AUC = 0.9761779952239501, Accuracy = 0.948905109489051,           Precision_1 = 0.946236559139785, Precision_0 = 0.9502762430939227          Recall_1 = 0.9072164948453608, Recall_0 = 0.9717514124293786, F1Score = 0.9263157894736843, kappa = 0.8872361690869539, MCC = 0.8876970100346031
Fold = 2, AUC = 0.9746053934416682, Accuracy = 0.9343065693430657,           Precision_1 = 0.898989898989899, Precision_0 = 0.9542857142857143          Recall_1 = 0.9175257731958762, Recall_0 = 0.943502824858757, F1Score = 0.9081632653061225, kappa = 0.857035190445823, MCC = 0.8571433398521561
Fold = 3, AUC = 0.962024579183412, Accuracy = 0.927007299270073,           Precision_1 = 0.8888888888888888, Pre

In [7]:
fold_metrics = pd.DataFrame(columns=['Accuracy','AUC','Precision_1','Precision_0','Recall_1','Recall_0','F1score','Kappa','MCC'])
fold_metrics['Accuracy'] = np.array(accuracies)
fold_metrics['AUC'] = np.array(aucs)
fold_metrics['Precision_1'] = np.array(precisions_1)
fold_metrics['Precision_0'] = np.array(precisions_0)
fold_metrics['Recall_1'] = np.array(recalls_1)
fold_metrics['Recall_0'] = np.array(recalls_0)
fold_metrics['F1score'] = np.array(f1scores)
fold_metrics['Kappa'] = np.array(kappas)
fold_metrics['MCC'] = np.array(MCCs)
fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.948905,0.965345,0.936842,0.955307,0.917526,0.966102,0.927083,0.887764,0.887878
1,0.948905,0.976178,0.946237,0.950276,0.907216,0.971751,0.926316,0.887236,0.887697
2,0.934307,0.974605,0.898990,0.954286,0.917526,0.943503,0.908163,0.857035,0.857143
3,0.927007,0.962025,0.888889,0.948571,0.907216,0.937853,0.897959,0.841150,0.841256
4,0.941606,0.970062,0.900990,0.965318,0.938144,0.943503,0.919192,0.873507,0.873944
5,0.956204,0.980546,0.938144,0.966102,0.938144,0.966102,0.938144,0.904246,0.904246
6,0.941606,0.975246,0.917526,0.954802,0.917526,0.954802,0.917526,0.872328,0.872328
7,0.941606,0.973965,0.945055,0.939891,0.886598,0.971751,0.914894,0.870518,0.871546
8,0.930657,0.979927,0.888889,0.954286,0.916667,0.938202,0.902564,0.848760,0.849002
9,0.963370,0.978107,0.967391,0.961326,0.927083,0.983051,0.946809,0.918895,0.919379


In [8]:
fold_metrics.loc[10,:] = [np.mean(np.array(accuracies)), np.mean(np.array(aucs)), np.mean(np.array(precisions_1)),
                               np.mean(np.array(precisions_0)), np.mean(np.array(recalls_1)), np.mean(np.array(recalls_0)),
                            np.mean(np.array(f1scores)), np.mean(np.array(kappas)), np.mean(np.array(MCCs))]

fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.948905,0.965345,0.936842,0.955307,0.917526,0.966102,0.927083,0.887764,0.887878
1,0.948905,0.976178,0.946237,0.950276,0.907216,0.971751,0.926316,0.887236,0.887697
2,0.934307,0.974605,0.898990,0.954286,0.917526,0.943503,0.908163,0.857035,0.857143
3,0.927007,0.962025,0.888889,0.948571,0.907216,0.937853,0.897959,0.841150,0.841256
4,0.941606,0.970062,0.900990,0.965318,0.938144,0.943503,0.919192,0.873507,0.873944
5,0.956204,0.980546,0.938144,0.966102,0.938144,0.966102,0.938144,0.904246,0.904246
6,0.941606,0.975246,0.917526,0.954802,0.917526,0.954802,0.917526,0.872328,0.872328
7,0.941606,0.973965,0.945055,0.939891,0.886598,0.971751,0.914894,0.870518,0.871546
8,0.930657,0.979927,0.888889,0.954286,0.916667,0.938202,0.902564,0.848760,0.849002
9,0.963370,0.978107,0.967391,0.961326,0.927083,0.983051,0.946809,0.918895,0.919379


In [9]:
fold_metrics.loc[11,:] = [np.std(np.array(accuracies)), np.std(np.array(aucs)), np.std(np.array(precisions_1)),
                               np.std(np.array(precisions_0)), np.std(np.array(recalls_1)), np.std(np.array(recalls_0)),
                            np.std(np.array(f1scores)), np.std(np.array(kappas)), np.std(np.array(MCCs))]

fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.948905,0.965345,0.936842,0.955307,0.917526,0.966102,0.927083,0.887764,0.887878
1,0.948905,0.976178,0.946237,0.950276,0.907216,0.971751,0.926316,0.887236,0.887697
2,0.934307,0.974605,0.898990,0.954286,0.917526,0.943503,0.908163,0.857035,0.857143
3,0.927007,0.962025,0.888889,0.948571,0.907216,0.937853,0.897959,0.841150,0.841256
4,0.941606,0.970062,0.900990,0.965318,0.938144,0.943503,0.919192,0.873507,0.873944
5,0.956204,0.980546,0.938144,0.966102,0.938144,0.966102,0.938144,0.904246,0.904246
6,0.941606,0.975246,0.917526,0.954802,0.917526,0.954802,0.917526,0.872328,0.872328
7,0.941606,0.973965,0.945055,0.939891,0.886598,0.971751,0.914894,0.870518,0.871546
8,0.930657,0.979927,0.888889,0.954286,0.916667,0.938202,0.902564,0.848760,0.849002
9,0.963370,0.978107,0.967391,0.961326,0.927083,0.983051,0.946809,0.918895,0.919379


In [10]:
fold_metrics.index = ['Fold_0','Fold_1','Fold_2','Fold_3','Fold_4','Fold_5','Fold_6','Fold_7','Fold_8','Fold_9','Mean','Std']
fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
Fold_0,0.948905,0.965345,0.936842,0.955307,0.917526,0.966102,0.927083,0.887764,0.887878
Fold_1,0.948905,0.976178,0.946237,0.950276,0.907216,0.971751,0.926316,0.887236,0.887697
Fold_2,0.934307,0.974605,0.898990,0.954286,0.917526,0.943503,0.908163,0.857035,0.857143
Fold_3,0.927007,0.962025,0.888889,0.948571,0.907216,0.937853,0.897959,0.841150,0.841256
Fold_4,0.941606,0.970062,0.900990,0.965318,0.938144,0.943503,0.919192,0.873507,0.873944
Fold_5,0.956204,0.980546,0.938144,0.966102,0.938144,0.966102,0.938144,0.904246,0.904246
Fold_6,0.941606,0.975246,0.917526,0.954802,0.917526,0.954802,0.917526,0.872328,0.872328
Fold_7,0.941606,0.973965,0.945055,0.939891,0.886598,0.971751,0.914894,0.870518,0.871546
Fold_8,0.930657,0.979927,0.888889,0.954286,0.916667,0.938202,0.902564,0.848760,0.849002
Fold_9,0.963370,0.978107,0.967391,0.961326,0.927083,0.983051,0.946809,0.918895,0.919379


# Final Model

In [11]:
X = data.iloc[:,:-2]
y = data.iloc[:,-2]

In [ ]:
best_params = {'colsample_bytree': 0.9611638174070645, 
 'gamma': 0.5344801495975495, 
 'learning_rate': 0.048882041230470526, 
 'max_depth': 14.0, 
 'min_child_weight': 0.0, 
 'reg_alpha': 0.0, 
 'reg_lambda': 1.164764580743543}

In [ ]:
model = XGBClassifier(n_jobs = -1, objective='binary:logistic',max_depth = int(best_params['max_depth']), 
                    gamma = best_params['gamma'],reg_lambda = best_params['reg_lambda'],
                    reg_alpha = int(best_params['reg_alpha']), min_child_weight=int(best_params['min_child_weight']),
                    colsample_bytree=best_params['colsample_bytree'],
                    learning_rate= best_params['learning_rate'])

model.fit(X, y)

In [ ]:
joblib.dump(model, 'XGB_Final.pkl')